 **Few shot text generation with T5 Transformer**

Author: Ramsri Goutham Golla

Linkedin : https://www.linkedin.com/in/ramsrig/

Twitter: https://twitter.com/ramsri_goutham

## 1. Install libraries

In [ ]:
!pip install transformers==2.9.0

     |████████████████████████████████| 645kB 13.6MB/s 
     |████████████████████████████████| 5.6MB 16.6MB/s 
     |████████████████████████████████| 1.2MB 55.8MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1e291769aaa0e2e968d0a2f18fbce53315bd66703bb52cd0a997ac58101bc121
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Mon Mar 29 04:02:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Prepare Model

In [2]:

import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    OPTForCausalLM, AutoTokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
opt = OPTForCausalLM.from_pretrained("facebook/opt-1.3b")


In [9]:
opt

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerNorm((204

In [31]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in opt.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in opt.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)



c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
# dataset preparation

true_false_adjective_tuples = [
                               ("The cat is alive","The cat is dead"),
                               ("The old woman is beautiful","The old woman is ugly"),
                               ("The purse is cheap","The purse is expensive"),
                               ("Her hair is curly","Her hair is straight"),
                               ("The bathroom is clean","The bathroom is dirty"),
                               ("The exam was easy","The exam was difficult"),
                               ("The house is big","The house is small"),
                               ("The house owner is good","The house owner is bad"),
                               ("The little kid is fat","The little kid is thin"),
                               ("She arrived early","She arrived late."),
                               ("John is very hardworking","John is very lazy"),
                               ("The fridge is empty","The fridge is full")

]

## 3. Train Loop

In [34]:
opt.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in true_false_adjective_tuples:
    input_sent = "falsify: "+input+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = opt(input_ids=input_ids, labels=lm_labels,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


epoch  0


c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


## 4. Test model

In [35]:
test_sent = 'falsify: The sailor was happy and joyful. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

opt.eval()
beam_outputs = opt.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

falsify: The sailor was happy and joyful. 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\HADOOP\AppData\Local\Temp\ipykernel_17520\743309940.py:18 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\HADOOP\\AppData\\Local\\Temp\\ipykernel_17520\\743309940.py'                         │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils_base.py:3468  │
│ in decode                                                                                        │
│                                                                                                  │
│   3465 │   │   # Convert inputs to python lists                                                  │
│   3466 │   │   token_ids = to_py_obj(token_ids)                                                  │
│   3467 │   │                                                                                     │
│ ❱ 3468 │   │   return self._decode(                                                              │
│   3469 │   │   │   token_ids=token_ids,                                                          │
│   3470 │   │   │   skip_special_tokens=skip_special_tokens,                                      │
│   3471 │   │   │   clean_up_tokenization_spaces=clean_up_tokenization_spaces,                    │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils.py:949 in     │
│ _decode                                                                                          │
│                                                                                                  │
│   946 │   │   │   else:                                                                          │
│   947 │   │   │   │   current_sub_text.append(token)                                             │
│   948 │   │   if current_sub_text:                                                               │
│ ❱ 949 │   │   │   sub_texts.append(self.convert_tokens_to_string(current_sub_text))              │
│   950 │   │                                                                                      │
│   951 │   │   if spaces_between_special_tokens:                                                  │
│   952 │   │   │   text = " ".join(sub_texts)                                                     │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\models\gpt2\tokenization_gpt2.py │
│ :316 in convert_tokens_to_string                                                                 │
│                                                                                                  │
│   313 │                                                                                          │
│   314 │   def convert_tokens_to_string(self, tokens):                                            │
│   315 │   │   """Converts a sequence of tokens (string) in a single string."""                   │
│ ❱ 316 │   │   text = "".join(tokens)                                                             │
│   317 │   │   text = bytearray([self.byte_decoder[c] for c in text]).decode("utf-8", errors=se   │
│   318 │   │   return text                                                                        │
│   319                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: sequence item 30: expected str instance, NoneType 

In [36]:
test_sent = 'falsify: This is a safe neighbourhood. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

opt.eval()
beam_outputs = opt.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

falsify: This is a safe neighbourhood. 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\HADOOP\AppData\Local\Temp\ipykernel_17520\2293813077.py:18 in <module>                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\HADOOP\\AppData\\Local\\Temp\\ipykernel_17520\\2293813077.py'                        │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils_base.py:3468  │
│ in decode                                                                                        │
│                                                                                                  │
│   3465 │   │   # Convert inputs to python lists                                                  │
│   3466 │   │   token_ids = to_py_obj(token_ids)                                                  │
│   3467 │   │                                                                                     │
│ ❱ 3468 │   │   return self._decode(                                                              │
│   3469 │   │   │   token_ids=token_ids,                                                          │
│   3470 │   │   │   skip_special_tokens=skip_special_tokens,                                      │
│   3471 │   │   │   clean_up_tokenization_spaces=clean_up_tokenization_spaces,                    │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils.py:949 in     │
│ _decode                                                                                          │
│                                                                                                  │
│   946 │   │   │   else:                                                                          │
│   947 │   │   │   │   current_sub_text.append(token)                                             │
│   948 │   │   if current_sub_text:                                                               │
│ ❱ 949 │   │   │   sub_texts.append(self.convert_tokens_to_string(current_sub_text))              │
│   950 │   │                                                                                      │
│   951 │   │   if spaces_between_special_tokens:                                                  │
│   952 │   │   │   text = " ".join(sub_texts)                                                     │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\models\gpt2\tokenization_gpt2.py │
│ :316 in convert_tokens_to_string                                                                 │
│                                                                                                  │
│   313 │                                                                                          │
│   314 │   def convert_tokens_to_string(self, tokens):                                            │
│   315 │   │   """Converts a sequence of tokens (string) in a single string."""                   │
│ ❱ 316 │   │   text = "".join(tokens)                                                             │
│   317 │   │   text = bytearray([self.byte_decoder[c] for c in text]).decode("utf-8", errors=se   │
│   318 │   │   return text                                                                        │
│   319                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: sequence item 30: expected str instance, NoneType 

In [28]:
test_sent = 'falsify: The tortoise was very slow. </s>'
test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

test_input_ids  = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]

opt.eval()
beam_outputs = opt.generate(
    input_ids=test_input_ids,attention_mask=test_attention_mask,
    max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\HADOOP\AppData\Local\Temp\ipykernel_17520\4128596348.py:18 in <module>                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\HADOOP\\AppData\\Local\\Temp\\ipykernel_17520\\4128596348.py'                        │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils_base.py:3468  │
│ in decode                                                                                        │
│                                                                                                  │
│   3465 │   │   # Convert inputs to python lists                                                  │
│   3466 │   │   token_ids = to_py_obj(token_ids)                                                  │
│   3467 │   │                                                                                     │
│ ❱ 3468 │   │   return self._decode(                                                              │
│   3469 │   │   │   token_ids=token_ids,                                                          │
│   3470 │   │   │   skip_special_tokens=skip_special_tokens,                                      │
│   3471 │   │   │   clean_up_tokenization_spaces=clean_up_tokenization_spaces,                    │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\tokenization_utils.py:949 in     │
│ _decode                                                                                          │
│                                                                                                  │
│   946 │   │   │   else:                                                                          │
│   947 │   │   │   │   current_sub_text.append(token)                                             │
│   948 │   │   if current_sub_text:                                                               │
│ ❱ 949 │   │   │   sub_texts.append(self.convert_tokens_to_string(current_sub_text))              │
│   950 │   │                                                                                      │
│   951 │   │   if spaces_between_special_tokens:                                                  │
│   952 │   │   │   text = " ".join(sub_texts)                                                     │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\transformers\models\gpt2\tokenization_gpt2.py │
│ :316 in convert_tokens_to_string                                                                 │
│                                                                                                  │
│   313 │                                                                                          │
│   314 │   def convert_tokens_to_string(self, tokens):                                            │
│   315 │   │   """Converts a sequence of tokens (string) in a single string."""                   │
│ ❱ 316 │   │   text = "".join(tokens)                                                             │
│   317 │   │   text = bytearray([self.byte_decoder[c] for c in text]).decode("utf-8", errors=se   │
│   318 │   │   return text                                                                        │
│   319                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: sequence item 31: expected str instance, NoneType 